In [1]:
import os
import rasterio as rio
import numpy as np
import shapely
import pyproj
import geopandas as gpd
import matplotlib.pyplot as plt
import rioxarray as riox
import rasterio as rio
import xarray as xr
import netCDF4
from osgeo import gdal
import pandas as pd
from datetime import timedelta
from datetime import datetime
import dask.array

import sys
sys.path.append('../')
import snowFun

In [2]:
# define folder and file paths
folder_AGVA = os.path.join('C:',os.sep,'Users','lzell','OneDrive - Colostate','Desktop',"AGVA")
folder_dems = os.path.join(folder_AGVA, "DEMs", "time_varying_DEMs", "10m")
folder_class = os.path.join(folder_AGVA, 'classified images', 'S2_Classified_Cloudmasked_Merged')
folder_meta = os.path.join(folder_AGVA, "classified images", "meta csv", "S2")
folder_mask = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Masks')
folder_debris = os.path.join(folder_AGVA, 'debris cover', 'raster')

# open rgi
path_rgi = os.path.join(folder_AGVA, 'RGI', "rgi_2km_o3regions", "rgi_2km_o3regions.shp")
rgi_gdf = gpd.read_file(path_rgi)

In [3]:
### choose if you want to do only the 45 validation glaciers
validation_only = 0

# load rgi names that have been saved to the classified folder
rgis_folder = list(set( [ i[3:17] for i in os.listdir(folder_class) if i!='merged.vrt' ] ))

# open list of validation glaciers
all_validation_df = pd.read_csv(os.path.join(folder_AGVA, 'Validation', 'Validation Glaciers.csv'))

# get rgi names for given o2 region
rgis_o2 = rgi_gdf[rgi_gdf['O2Region']=='4']['RGIId'].values

# select which rgis to analyze
if validation_only:
    rgis_to_analyze = list( set(rgis_folder).intersection(set(all_validation_df['RGIId'].values)) )
else:
    # rgis_to_analyze = ["RGI60-01.09162"] # just a single rgi
    rgis_to_analyze = rgis_folder # everything that is available
#     rgis_to_analyze = list( set(rgis_folder).intersection(set(rgis_o2)) ) # all the rgis in the folder than are in this o2region

# get list of glacier area for each rgi
areas = [rgi_gdf[rgi_gdf['RGIId']==i]['Area'].values for i in rgis_to_analyze]

# make df
rgis_to_analyze_df = pd.DataFrame({"RGIId":rgis_to_analyze, 'Area':areas})

# sort however you want
rgis_to_analyze_df = rgis_to_analyze_df.sort_values('Area')

# grab rgi names
rgis_to_analyze = rgis_to_analyze_df['RGIId'].values

print(len(rgis_to_analyze_df))
# print(rgis_to_analyze[:10])
# print(rgis_to_analyze_df[:10])

3031


In [ ]:
c = 0
for i in range(len(rgis_to_analyze)):
#     if c>0: continue
#     if i<43: continue
    # subset rgi to single outline, by choosing rgiid or rgi name
    rgiid = rgis_to_analyze[i]
#     if rgiid!= "RGI60-01.15731": continue #10910 08989

    # quickly grab glacier area
    ga = rgi_gdf[rgi_gdf['RGIId']==rgiid]['Area'].values[0]

#     if ga<100: continue
        
    # set folder
    if validation_only:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2', 'Validation')
    else:
        folder_save = os.path.join(folder_AGVA, 'Derived products', 'S2')
       
    # check if this glacier has been run already, skip if so
    temp_path = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_2022_daily_AAs_shadowed.nc")
    if os.path.exists(temp_path): continue
        
    # check if the initial products we need exist yet. if not, then continue
    path_1 = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_2022_daily_AAs.nc")
    path_2 = os.path.join(folder_save, 'Shadows', f"S2_{rgiid}_2022_daily_shadows.nc")
    if not os.path.exists(path_1): continue
    if not os.path.exists(path_2): continue
    
    # print progress
    print(f"\nStarting {i+1} of {len(rgis_to_analyze)}: {rgiid}  {ga} km2")
    
    # grab just this rgi geometry and info
    rgi_single = rgi_gdf[rgi_gdf['RGIId']==rgiid].to_crs("EPSG:3338")
    single_geometry = rgi_single.geometry

    # single_geometry = single_geometry.buffer(-100) #what if we buffer out the exterior 100 meters of the glacier
 
    # open glacier mask, count how many pixels there are
    glacier_mask = xr.open_dataset(os.path.join(folder_mask, f"S2_{rgiid}_mask.nc"), chunks='auto').glacier
    
    # open the observed faction df
#     obs_path = os.path.join(folder_save, 'Daily AAs', 'observed', f"S2_{rgiid}_observed.csv")
#     obs_df = pd.read_csv(obs_path)
    
    # for each year, open the daily data, coarsen and resave
    for y in [2018,2019,2020,2021,2022]:
        if ga>100: print(y)
#         if y!=2018: continue
        
        # open data
        path_open = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs.nc")
        path_shadow = os.path.join(folder_save, 'Shadows', f"S2_{rgiid}_{y}_daily_shadows.nc")
        path_debris = os.path.join(folder_debris, f"{rgiid}_debris.tif")
        
        # if small glacier, we dont need to chunk
        if ga>1000:
            snow = xr.open_dataset(path_open, chunks={'time':1})
            shadows = xr.open_dataset(path_shadow, chunks={'time':1})
            debris = riox.open_rasterio(path_debris)
        elif ga>100:
            snow = xr.open_dataset(path_open, chunks={'time':10})
            shadows = xr.open_dataset(path_shadow, chunks={'time':10})
            debris = riox.open_rasterio(path_debris)
        else:
            snow = xr.open_dataset(path_open)
            shadows = xr.open_dataset(path_shadow)
            debris = riox.open_rasterio(path_debris)
        
        # at this point:
        # snow=0 is off-glacier, shadow, cloud
        # snow=1 is snow
        # snow=2,3,4,6 is firn,ice,debris,water (in that order) (5 is shadow)
        # terrain shadow mask has 1=usable, 0=shadow or off-glacier
        # debris mask has 1=debris, 0=not debris
        
        # apply debris mask. where 'snow' is not in terrain shadow and is classified as shadow, ice, or firn, but should be debris
        # make snow 4 where (debris=1) and (shadow=1) and (snow=0 or 2 or 3)
        snow = xr.where( (debris==1) & (shadows==1) & (snow.isin([2,3,5])), 4, snow)
        
#         # apply terrain shadow mask
#         # make snow 0 where shadows=0
        snow = snow.where(shadows==1, 0)
        
#         # make all shadows 0
        snow = snow.where(snow!=5,0)

#         # reapply mask
        snow = snow.where(glacier_mask>0.5,0).astype('uint8')
     
        # save
        path_temp = os.path.join(folder_save, 'Daily AAs', f"S2_{rgiid}_{y}_daily_AAs_shadowed.nc")
        encoding = {"class":{"zlib": True, "dtype": "uint8"}}
        snow.to_netcdf(path_temp, encoding=encoding)
     
    c+=1
print("Done!")
# should start at 1953, 6.217 km2


Starting 1953 of 3031: RGI60-01.03733  6.217 km2

Starting 1955 of 3031: RGI60-01.00962  6.225 km2

Starting 1956 of 3031: RGI60-01.12187  6.229 km2

Starting 1957 of 3031: RGI60-01.14059  6.232 km2

Starting 1958 of 3031: RGI60-01.02908  6.236 km2

Starting 1960 of 3031: RGI60-01.16530  6.265 km2

Starting 1961 of 3031: RGI60-01.05118  6.266 km2

Starting 1962 of 3031: RGI60-01.17477  6.27 km2

Starting 1964 of 3031: RGI60-01.04425  6.275 km2

Starting 1965 of 3031: RGI60-01.17616  6.276 km2

Starting 1966 of 3031: RGI60-01.03171  6.282 km2

Starting 1967 of 3031: RGI60-01.01756  6.295 km2

Starting 1968 of 3031: RGI60-01.08071  6.31 km2

Starting 1969 of 3031: RGI60-01.15176  6.344 km2

Starting 1970 of 3031: RGI60-01.16722  6.345 km2

Starting 1971 of 3031: RGI60-01.16309  6.348 km2

Starting 1973 of 3031: RGI60-01.01436  6.38 km2

Starting 1974 of 3031: RGI60-01.08778  6.38 km2

Starting 1975 of 3031: RGI60-01.15719  6.401 km2

Starting 1977 of 3031: RGI60-01.21721  6.422 km2

Sta


Starting 2183 of 3031: RGI60-01.01337  8.548 km2

Starting 2184 of 3031: RGI60-01.18143  8.592 km2

Starting 2185 of 3031: RGI60-01.03684  8.635 km2

Starting 2186 of 3031: RGI60-01.09492  8.638 km2

Starting 2188 of 3031: RGI60-01.16993  8.649 km2

Starting 2189 of 3031: RGI60-01.15190  8.652 km2

Starting 2190 of 3031: RGI60-01.09201  8.66 km2

Starting 2191 of 3031: RGI60-01.03790  8.679 km2

Starting 2194 of 3031: RGI60-01.05916  8.701 km2

Starting 2195 of 3031: RGI60-01.05263  8.714 km2

Starting 2196 of 3031: RGI60-01.04762  8.737 km2

Starting 2197 of 3031: RGI60-01.03633  8.747 km2

Starting 2199 of 3031: RGI60-01.01401  8.792 km2

Starting 2200 of 3031: RGI60-01.13950  8.795 km2

Starting 2201 of 3031: RGI60-01.15356  8.799 km2

Starting 2202 of 3031: RGI60-01.01745  8.81 km2

Starting 2203 of 3031: RGI60-01.16312  8.819 km2

Starting 2204 of 3031: RGI60-01.13958  8.832 km2

Starting 2205 of 3031: RGI60-01.13980  8.838 km2

Starting 2207 of 3031: RGI60-01.01403  8.885 km2

S


Starting 2387 of 3031: RGI60-01.06747  12.69 km2

Starting 2389 of 3031: RGI60-01.01328  12.76 km2

Starting 2390 of 3031: RGI60-01.05976  12.764 km2

Starting 2391 of 3031: RGI60-01.03787  12.869 km2

Starting 2392 of 3031: RGI60-01.02657  12.892 km2

Starting 2393 of 3031: RGI60-01.03560  12.933 km2

Starting 2394 of 3031: RGI60-01.04743  13.013 km2

Starting 2395 of 3031: RGI60-01.03792  13.049 km2

Starting 2396 of 3031: RGI60-01.17995  13.064 km2

Starting 2397 of 3031: RGI60-01.12180  13.142 km2

Starting 2398 of 3031: RGI60-01.03549  13.2 km2

Starting 2399 of 3031: RGI60-01.15779  13.222 km2

Starting 2400 of 3031: RGI60-01.13478  13.241 km2

Starting 2402 of 3031: RGI60-01.09532  13.263 km2

Starting 2404 of 3031: RGI60-01.08772  13.289 km2

Starting 2405 of 3031: RGI60-01.00027  13.29 km2

Starting 2406 of 3031: RGI60-01.09693  13.353 km2

Starting 2408 of 3031: RGI60-01.15689  13.448 km2

Starting 2409 of 3031: RGI60-01.10928  13.507 km2

Starting 2410 of 3031: RGI60-01.096


Starting 2603 of 3031: RGI60-01.14375  23.418 km2

Starting 2604 of 3031: RGI60-01.10955  23.501 km2

Starting 2605 of 3031: RGI60-01.02629  23.591 km2

Starting 2606 of 3031: RGI60-01.18032  23.815 km2

Starting 2607 of 3031: RGI60-01.10108  23.944 km2

Starting 2608 of 3031: RGI60-01.07020  24.114 km2

Starting 2609 of 3031: RGI60-01.05343  24.141 km2

Starting 2612 of 3031: RGI60-01.17771  24.315 km2

Starting 2613 of 3031: RGI60-01.08395  24.354 km2

Starting 2614 of 3031: RGI60-01.02689  24.396 km2

Starting 2615 of 3031: RGI60-01.17619  24.438 km2

Starting 2616 of 3031: RGI60-01.01424  24.482 km2

Starting 2617 of 3031: RGI60-01.09860  24.735 km2

Starting 2618 of 3031: RGI60-01.11788  24.781 km2

Starting 2621 of 3031: RGI60-01.01735  24.925 km2

Starting 2622 of 3031: RGI60-01.05880  24.984 km2

Starting 2623 of 3031: RGI60-01.12369  25.072 km2

Starting 2625 of 3031: RGI60-01.13947  25.206 km2

Starting 2626 of 3031: RGI60-01.18748  25.312 km2

Starting 2627 of 3031: RGI60-0


Starting 2828 of 3031: RGI60-01.04972  67.31 km2

Starting 2830 of 3031: RGI60-01.15770  68.095 km2

Starting 2831 of 3031: RGI60-01.18055  68.328 km2

Starting 2835 of 3031: RGI60-01.16149  71.669 km2

Starting 2836 of 3031: RGI60-01.19542  71.722 km2

Starting 2837 of 3031: RGI60-01.05886  71.741 km2

Starting 2838 of 3031: RGI60-01.04591  71.753 km2

Starting 2839 of 3031: RGI60-01.12735  73.96 km2

Starting 2840 of 3031: RGI60-01.17880  74.85 km2

Starting 2843 of 3031: RGI60-01.00576  77.87 km2

Starting 2844 of 3031: RGI60-01.17464  78.043 km2

Starting 2845 of 3031: RGI60-01.09148  78.446 km2

Starting 2846 of 3031: RGI60-01.09810  78.889 km2

Starting 2848 of 3031: RGI60-01.13932  79.452 km2

Starting 2850 of 3031: RGI60-01.13789  79.665 km2

Starting 2852 of 3031: RGI60-01.20841  80.284 km2

Starting 2853 of 3031: RGI60-01.17481  80.413 km2

Starting 2854 of 3031: RGI60-01.13794  81.439 km2

Starting 2856 of 3031: RGI60-01.01513  81.626 km2

Starting 2858 of 3031: RGI60-01.05